# Experiment 3: Variance
This notebook tests the impact of data augmentation and fine tuning on the final results. We notice that data augmentation and fine tuning inject noise into the process. 

Tests:
- Zero-shot Transfer Learning
- Fine tuned Transfer Learning without data augmentation
- Fine Tuned Transfer Learning with data augmentation

Datasets:
- OpenSARShip
- FUSAR-Ship

In [1]:
#Check the GPU and RAM
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Jan 25 21:14:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Mount drive and install python packages
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/MyDrive/SAR_BAL')

Mounted at /content/drive


In [3]:
!pip install graphlearning annoy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for graphlearning: filename=graphlearning-1.2.1-cp38-cp38-linux_x86_64.whl size=158942 sha256=c0443ae286b4bb217bfefbaf17e94c38a5b304519427d6fee948e92e619ade71
  Stored in directory: /root/.cache/pip/wheels/5e/84/62/681cf2c2a59f1f5e58d52fb110e2eb4c93c9c808d932829543
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=581423 sha256=815db1cf1383d4b563f8370aeca17f4e5ad3a471ac420613e3f61de22225e914
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
  error: subprocess-exited-with-error
  
  × python setup.py bdist_w

In [4]:
import pandas as pd

import utils
from experiments import experiment_3, EXPERIMENT_3_SAVE_PATH, EXPERIMENT_3_NUM_EXPERIMENTS

## Parameters

In [5]:
#Non-Default Parameters

#(data augmentation, embedding)
test_list = [(False, 'zero_shot_tl'), (False, 'fine_tuned_tl'), (True, 'fine_tuned_tl')]

HARDWARE_ACCELERATION = True

## Experiments


In [6]:
all_results = {}

for (data_augmentation, embedding) in test_list:
    for dataset in utils.AVAILABLE_SAR_DATASETS[1:]:
        key = str(data_augmentation) + "_" + dataset + "_" + embedding
        print(key)
        experiment_result = experiment_3(
            dataset,
            embedding,
            data_augmentation,
            hardware_acceleration=HARDWARE_ACCELERATION,
        )
        all_results[key] = experiment_result.copy()

        df = pd.DataFrame.from_dict(experiment_result, orient='index')
        df.to_pickle(EXPERIMENT_3_SAVE_PATH + key + '.pkl')

        print(experiment_result)
        print()

False_open_sar_ship_zero_shot_tl


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.

{'num_points': 803, 'experiments': 20, 'mean': 81.02478231748158, 'std_dev': 0.7616514060088803, 'max': 82.31748158070998, 'data': array([81.31279303, 81.64768922, 80.57602143, 80.5090422 , 81.37977227,
       80.57602143, 82.31748158, 80.64300067, 79.37039518, 80.24112525,
       81.37977227, 81.44675151, 79.57133289, 82.18352311, 81.84862693,
       80.91091762, 80.97789685, 81.64768922, 80.57602143, 81.37977227])}

False_fusar_zero_shot_tl


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x0.5-f707e7126e.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x0.5-f707e7126e.pth


  0%|          | 0.00/5.28M [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.

{'num_points': 3302, 'experiments': 20, 'mean': 88.57142857142857, 'std_dev': 0.3491546966859851, 'max': 89.25353925353926, 'data': array([88.93178893, 88.41698842, 88.86743887, 87.9021879 , 88.73873874,
       88.22393822, 88.61003861, 88.22393822, 88.35263835, 88.0952381 ,
       88.67438867, 88.996139  , 88.67438867, 88.0952381 , 89.06048906,
       88.73873874, 88.48133848, 88.41698842, 88.67438867, 89.25353925])}

False_open_sar_ship_fine_tuned_tl
Training model for 30 epochs
Training complete in 0m 3s
Best val Acc: 0.496561
Training model for 30 epochs
Training complete in 0m 2s
Best val Acc: 0.502522
Training model for 30 epochs
Training complete in 0m 2s
Best val Acc: 0.497937
Training model for 30 epochs
Training complete in 0m 3s
Best val Acc: 0.520403
Training model for 30 epochs
Training complete in 0m 3s
Best val Acc: 0.500688
Training model for 30 epochs
Training complete in 0m 3s
Best val Acc: 0.496103
Training model for 30 epochs
Training complete in 0m 3s
Best val Acc: